# Mnist classification demo

In [1]:
%%capture
import grpc 
import numpy as np
import hydro_serving_grpc as hs
import keras
from keras.datasets import mnist
from keras import backend as K
import os

## Read data sample

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_rows, img_cols = 28, 28
num_classes = 10

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## Setup channel

In [6]:
channel = grpc.insecure_channel("localhost:9090")
stub = hs.PredictionServiceStub(channel) 
model_spec = hs.ModelSpec(name="mnist2", signature_name="INFERRED_PIPELINE_SIGNATURE")

## Create tensors

In [7]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=1), hs.TensorShapeProto.Dim(size=28), hs.TensorShapeProto.Dim(size=28)])
tensor = hs.TensorProto(dtype=hs.DT_DOUBLE, tensor_shape=tensor_shape, double_val=np.array([x_test[0]]).flatten().astype(np.float32))

## Send request to model servable

In [11]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"input": tensor}) 
result = stub.Predict(request)
print('Output values:')
print(result.outputs['probabilities'])
print('Ground truth label:')
print(y_test[0])

Output values:
dtype: DT_DOUBLE
tensor_shape {
  dim {
    size: -1
  }
  dim {
    size: 10
  }
}
double_val: 2.9983486456330866e-05
double_val: 1.495610413293491e-09
double_val: 4.615424040821381e-05
double_val: 0.0024091629311442375
double_val: 1.2768715578204137e-06
double_val: 2.4745755581534468e-05
double_val: 7.330490880974594e-09
double_val: 0.996881365776062
double_val: 2.5152998205157928e-05
double_val: 0.0005822270177304745

Ground truth label:
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
